# 2024 Day 12: Garden Groups
Why not search for the Chief Historian near the gardener and his massive farm? There's plenty of food, so The Historians grab something to eat while they search.

You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.

Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a region. For example:

```
AAAA
BBCD
BBCC
EEEC
```
This 4x4 arrangement includes garden plots growing five different types of plants (labeled A, B, C, D, and E), each grouped into their own region.

In order to accurately calculate the cost of the fence around a single region, you need to know that region's area and perimeter.

The area of a region is simply the number of garden plots the region contains. The above map's type A, B, and C plants are each in a region of area 4. The type E plants are in a region of area 3; the type D plants are in a region of area 1.

Each garden plot is a square and so has four sides. The perimeter of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type A and C plants are each in a region with perimeter 10. The type B and E plants are each in a region with perimeter 8. The lone D plot forms its own region with perimeter 4.

Visually indicating the sides of plots in each region that contribute to the perimeter using - and |, the above map's regions' perimeters are measured as follows:

```
+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
```

Plants of the same type can appear in multiple separate regions, and regions can even appear within other regions. For example:

```
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
```

The above map contains five regions, one containing all of the O garden plots, and the other four each containing a single X plot.

The four X regions each have area 1 and perimeter 4. The region containing 21 type O plants is more complicated; in addition to its outer edge contributing a perimeter of 20, its boundary with each X region contributes an additional 4 to its perimeter, for a total perimeter of 36.

Due to "modern" business practices, the price of fence required for a region is found by multiplying that region's area by its perimeter. The total price of fencing all regions on a map is found by adding together the price of fence for every region on the map.

In the first example, region A has price 4 * 10 = 40, region B has price 4 * 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region E has price 3 * 8 = 24. So, the total price for the first example is 140.

In the second example, the region with all of the O plants has price 21 * 36 = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total price of 772 (756 + 4 + 4 + 4 + 4).

Here's a larger example:

```
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
```

It contains:

- region of R plants with price 12 * 18 = 216.
- region of I plants with price 4 * 8 = 32.
- region of C plants with price 14 * 28 = 392.
- region of F plants with price 10 * 18 = 180.
- region of V plants with price 13 * 20 = 260.
- region of J plants with price 11 * 20 = 220.
- region of C plants with price 1 * 4 = 4.
- region of E plants with price 13 * 18 = 234.
- region of I plants with price 14 * 22 = 308.
- region of M plants with price 5 * 12 = 60.
- region of S plants with price 3 * 8 = 24.

So, it has a total price of 1930.

What is the total price of fencing all regions on your map?

In [1]:
# Loading garden map (puzzle input) from file
with open('aoc-2024-day-12.txt', 'r') as f:
    raw_garden_map = f.read().splitlines()

# Setting the sample data
sample_data = '''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
'''.splitlines()

# Overriding the input for testing purposes
# (Note: Comment out the next line to use the actual puzzle input)
# raw_garden_map = sample_data

# Parsing the garden map into a 2D list
garden_map = [list(row) for row in raw_garden_map]

In [2]:
def check_next_locations(garden_map, x, y, current_region, next_to_visit, visited):
    """
    Check the next locations to visit from the current position.
    
    Inputs:
        - garden_map (list): 2D list representing the garden map
        - x (int): Current x-coordinate
        - y (int): Current y-coordinate
        - current_region (dict): The region information
        - next_to_visit (list): List of positions to visit next
        - visited (set): Set of already visited positions

    Returns:
        - next_to_visit (list): List of tuples representing the next positions to visit
        - visited (set): Updated set of visited positions
        - current_region (dict): Updated region information
    """

    # Marking the current position as visited
    visited.add((x, y))

    # Incrementing the count of the current region
    current_region['count'] += 1

    # Setting the directions to check
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    # Iterating over each direction
    for dx, dy in directions:

        # Getting the next location
        nx, ny = x + dx, y + dy

        

        # Checking if the next location is within bounds and matches the current region
        if 0 <= nx < len(garden_map) and 0 <= ny < len(garden_map[0]):
            if garden_map[nx][ny] == current_region['region']:
                if (nx, ny) not in visited:
                    next_to_visit.append((nx, ny))
                    visited.add((nx, ny))
            else:
                current_region['walls'] += 1
        else:
            current_region['walls'] += 1

    return next_to_visit, visited, current_region

In [3]:
# Instantiating a set for visited positions
visited = set()

# Instanitiating a list to hold the next locations to visit
next_to_visit = []

# Instantiating a list to hold the regions found
regions = []

# Iterating over all the positions in the garden map
for x in range(len(garden_map)):
    for y in range(len(garden_map[0])):

        # Checking to see if the current location has already been visited
        if (x, y) in visited:
            continue

        # Setting the current region information
        current_region = {
            'region': garden_map[x][y],
            'count': 0,
            'walls': 0
        }

        # Setting the current position in next to visit
        next_to_visit.append((x, y))

        # Iterating while there are still locations to visit
        while next_to_visit:

            # Checking for next positions to visit
            current_x, current_y = next_to_visit.pop(0)

            # Checking for the next locations to visit
            next_to_visit, visited, current_region = check_next_locations(
                garden_map, current_x, current_y, current_region, next_to_visit, visited
            )

        # Appending the current region to the list of regions found
        regions.append(current_region)

In [4]:
# Instantiating the total sum
total_sum = 0

# Iterating over each region found
for region in regions:
    # Calculating the product of count and walls for the region
    product = region['count'] * region['walls']
    
    # Adding the product to the total sum
    total_sum += product

print(f'The total sum of the products of counts and walls for all regions is: {total_sum}')

The total sum of the products of counts and walls for all regions is: 1477762


## Part 2

Fortunately, the Elves are trying to order so much fence that they qualify for a bulk discount!

Under the bulk discount, instead of using the perimeter to calculate the price, you need to use the number of sides each region has. Each straight section of fence counts as a side, regardless of how long it is.

Consider this example again:

```
AAAA
BBCD
BBCC
EEEC
```

The region containing type A plants has 4 sides, as does each of the regions containing plants of type B, D, and E. However, the more complex region containing the plants of type C has 8 sides!

Using the new method of calculating the per-region price by multiplying the region's area by its number of sides, regions A through E have prices 16, 16, 32, 4, and 12, respectively, for a total price of 80.

The second example above (full of type X and O plants) would have a total price of 436.

Here's a map that includes an E-shaped region full of type E plants:

```
EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
```

The E-shaped region has an area of 17 and 12 sides for a price of 204. Including the two regions full of type X plants, this map has a total price of 236.

This map has a total price of 368:

```
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
```

It includes two regions full of type B plants (each with 4 sides) and a single region full of type A plants (with 4 sides on the outside and 8 more sides on the inside, a total of 12 sides). Be especially careful when counting the fence around regions like the one full of type A plants; in particular, each section of fence has an in-side and an out-side, so the fence does not connect across the middle of the region (where the two B regions touch diagonally). (The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided.)

The larger example from before now has the following updated prices:

- region of R plants with price 12 * 10 = 120.
- region of I plants with price 4 * 4 = 16.
- region of C plants with price 14 * 22 = 308.
- region of F plants with price 10 * 12 = 120.
- region of V plants with price 13 * 10 = 130.
- region of J plants with price 11 * 12 = 132.
- region of C plants with price 1 * 4 = 4.
- region of E plants with price 13 * 8 = 104.
- region of I plants with price 14 * 16 = 224.
- region of M plants with price 5 * 6 = 30.
- region of S plants with price 3 * 6 = 18.

Adding these together produces its new total price of 1206.

What is the new total price of fencing all regions on your map?

In [5]:
# Loading garden map (puzzle input) from file
with open('aoc-2024-day-12.txt', 'r') as f:
    raw_garden_map = f.read().splitlines()

# Setting the sample data
sample_data = '''RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
'''.splitlines()

# Overriding the input for testing purposes
# (Note: Comment out the next line to use the actual puzzle input)
raw_garden_map = sample_data

# Parsing the garden map into a 2D list
garden_map = [list(row) for row in raw_garden_map]